[![Dataflowr](https://raw.githubusercontent.com/dataflowr/website/master/_assets/dataflowr_logo.png)](https://dataflowr.github.io/website/)

# Module 2: PyTorch tensors and automatic differentiation

[Video timestamp](https://youtu.be/BmAS8IH7n3c?t=103)

In [1]:
import matplotlib.pyplot as plt
import torch
import numpy as np

In [2]:
torch.__version__

'1.10.0'

Tensors are used to encode the signal to process, but also the internal states and parameters of models.

**Manipulating data through this constrained structure allows to use CPUs and GPUs at peak performance.**

Construct a 3x5 matrix, uninitialized:

In [13]:
x = torch.empty(3,5)
print(x.dtype)
print(x)

torch.float32
tensor([[9.2755e-39, 1.0561e-38, 8.9082e-39, 1.0469e-38, 9.2755e-39],
        [4.2246e-39, 1.0286e-38, 1.0653e-38, 1.0194e-38, 8.4490e-39],
        [1.0469e-38, 9.3674e-39, 9.9184e-39, 8.7245e-39, 9.2755e-39]])


If you got an error this [stackoverflow link](https://stackoverflow.com/questions/50617917/overflow-when-unpacking-long-pytorch) might be useful...

In [18]:
x = torch.randn(3,5)
print(x)

tensor([[ 1.2266,  0.6328,  1.4917,  0.6065,  0.8264],
        [-0.1950,  0.1709, -0.4795, -0.6010, -1.4236],
        [ 1.2294, -0.3296, -0.8624,  0.7584,  0.1417]])


In [19]:
print(x.size())

torch.Size([3, 5])


torch.Size is in fact a [tuple](https://docs.python.org/3/tutorial/datastructures.html#tuples-and-sequences), so it supports the same operations.

[Video timestamp](https://youtu.be/BmAS8IH7n3c?t=272)

In [20]:
x.size()[1]

5

In [26]:
x.size() == (3,5)

True

### Bridge to numpy

[Video timestamp](https://youtu.be/BmAS8IH7n3c?t=325)

In [42]:
y = x.numpy()
print(y)

[[ 1.2265537   0.6328068   1.4916524   0.60651654  0.8263821 ]
 [-0.19496761  0.17094342 -0.47948056 -0.60096526 -1.4236253 ]
 [ 1.2294048  -0.3296298  -0.86242515  0.75837725  0.14169262]]


In [43]:
a = np.ones(5)
b = torch.from_numpy(a)
print(a.dtype)
print(b)

float64
tensor([1., 1., 1., 1., 1.], dtype=torch.float64)


In [44]:
c = b.long()
print(c.dtype, c)
print(b.dtype, b)

torch.int64 tensor([1, 1, 1, 1, 1])
torch.float64 tensor([1., 1., 1., 1., 1.], dtype=torch.float64)


In [45]:
xr = torch.randn(3, 5)
print(xr.dtype, xr)

torch.float32 tensor([[-1.9006,  0.2213,  1.8395, -0.9421,  0.8637],
        [-0.9052,  0.6989,  0.3385, -1.8417, -0.7456],
        [ 0.3636,  0.3216, -1.5531,  0.5260,  0.8010]])


In [46]:
resb = xr + b
resb

tensor([[-0.9006,  1.2213,  2.8395,  0.0579,  1.8637],
        [ 0.0948,  1.6989,  1.3385, -0.8417,  0.2544],
        [ 1.3636,  1.3216, -0.5531,  1.5260,  1.8010]], dtype=torch.float64)

In [47]:
resc = xr + c
resc

tensor([[-0.9006,  1.2213,  2.8395,  0.0579,  1.8637],
        [ 0.0948,  1.6989,  1.3385, -0.8417,  0.2544],
        [ 1.3636,  1.3216, -0.5531,  1.5260,  1.8010]])

Be careful with types!

In [48]:
resb == resc

tensor([[ True,  True,  True,  True,  True],
        [ True, False, False,  True,  True],
        [False, False,  True, False, False]])

In [49]:
torch.set_printoptions(precision=10)

In [52]:
resb[1,1]

tensor(1.6989342570, dtype=torch.float64)

In [53]:
resc[1,1]

tensor(1.6989343166)

In [54]:
resc[0,1].dtype

torch.float32

In [55]:
xr[0,1]

tensor(0.2212979794)

In [56]:
torch.set_printoptions(precision=4)

### [Broadcasting](https://docs.scipy.org/doc/numpy-1.13.0/user/basics.broadcasting.html)

[Video timestamp](https://youtu.be/BmAS8IH7n3c?t=670)

Broadcasting automagically expands dimensions by replicating coefficients, when it is necessary to perform operations.

1. If one of the tensors has fewer dimensions than the other, it is reshaped by adding as many dimensions of size 1 as necessary in the front; then
2. for every mismatch, if one of the two tensor is of size one, it is expanded along this axis by replicating  coefficients.

If there is a tensor size mismatch for one of the dimension and neither of them is one, the operation fails.

In [57]:
A = torch.tensor([[1.], [2.], [3.], [4.]])
print(A.size())
B = torch.tensor([[5., -5., 5., -5., 5.]])
print(B.size())
C = A + B

torch.Size([4, 1])
torch.Size([1, 5])


In [58]:
C

tensor([[ 6., -4.,  6., -4.,  6.],
        [ 7., -3.,  7., -3.,  7.],
        [ 8., -2.,  8., -2.,  8.],
        [ 9., -1.,  9., -1.,  9.]])

The original (column-)vector
\begin{eqnarray*}
A = \left( \begin{array}{c}
1\\
2\\
3\\
4\\
\end{array}\right)
\end{eqnarray*}
is transformed into the matrix 
\begin{eqnarray*}
A = \left( \begin{array}{ccccc}
1&1&1&1&1\\
2&2&2&2&2\\
3&3&3&3&3\\
4&4&4&4&4
\end{array}\right)
\end{eqnarray*}
and the original (row-)vector
\begin{eqnarray*}
C = (5,-5,5,-5,5)
\end{eqnarray*}
is transformed into the matrix
\begin{eqnarray*}
C = \left( \begin{array}{ccccc}
5&-5&5&-5&5\\
5&-5&5&-5&5\\
5&-5&5&-5&5\\
5&-5&5&-5&5
\end{array}\right)
\end{eqnarray*}
so that summing these matrices gives:
\begin{eqnarray*}
A+C = \left( \begin{array}{ccccc}
6&-4&6&-4&6\\
7&-3&7&-3&7\\
8&-2&8&-2&8\\
9&-1&9&-1&9
\end{array}\right)
\end{eqnarray*}

### In-place modification

[Video timestamp](https://youtu.be/BmAS8IH7n3c?t=875)

In [59]:
x

tensor([[ 1.2266,  0.6328,  1.4917,  0.6065,  0.8264],
        [-0.1950,  0.1709, -0.4795, -0.6010, -1.4236],
        [ 1.2294, -0.3296, -0.8624,  0.7584,  0.1417]])

In [60]:
xr

tensor([[-1.9006,  0.2213,  1.8395, -0.9421,  0.8637],
        [-0.9052,  0.6989,  0.3385, -1.8417, -0.7456],
        [ 0.3636,  0.3216, -1.5531,  0.5260,  0.8010]])

In [61]:
print(x+xr)

tensor([[-0.6740,  0.8541,  3.3311, -0.3356,  1.6900],
        [-1.1002,  0.8699, -0.1409, -2.4426, -2.1693],
        [ 1.5930, -0.0080, -2.4155,  1.2844,  0.9427]])


In [62]:
x.add_(xr)
print(x)

tensor([[-0.6740,  0.8541,  3.3311, -0.3356,  1.6900],
        [-1.1002,  0.8699, -0.1409, -2.4426, -2.1693],
        [ 1.5930, -0.0080, -2.4155,  1.2844,  0.9427]])


Any operation that mutates a tensor in-place is post-fixed with an ```_```

For example: ```x.fill_(y)```, ```x.t_()```, will change ```x```.

In [66]:
print(x.t())

tensor([[-0.6740, -1.1002,  1.5930],
        [ 0.8541,  0.8699, -0.0080],
        [ 3.3311, -0.1409, -2.4155],
        [-0.3356, -2.4426,  1.2844],
        [ 1.6900, -2.1693,  0.9427]])


In [67]:
x.t_()
print(x)

tensor([[-0.6740, -1.1002,  1.5930],
        [ 0.8541,  0.8699, -0.0080],
        [ 3.3311, -0.1409, -2.4155],
        [-0.3356, -2.4426,  1.2844],
        [ 1.6900, -2.1693,  0.9427]])


### Shared memory

[Video timestamp](https://youtu.be/BmAS8IH7n3c?t=990)

Also be careful, changing the torch tensor modify the numpy array and vice-versa...

This is explained in the PyTorch documentation [here](https://pytorch.org/docs/stable/torch.html#torch.from_numpy):
The returned tensor by `torch.from_numpy` and ndarray share the same memory. Modifications to the tensor will be reflected in the ndarray and vice versa. 

In [69]:
a = np.ones(5)
b = torch.from_numpy(a)
print(b)

tensor([1., 1., 1., 1., 1.], dtype=torch.float64)


In [70]:
a[2] = 0
print(b)

tensor([1., 1., 0., 1., 1.], dtype=torch.float64)


In [71]:
b[3] = 5
print(a)

[1. 1. 0. 5. 1.]


### Cuda

[Video timestamp](https://youtu.be/BmAS8IH7n3c?t=1120)

In [73]:
torch.cuda.is_available()

False

In [74]:
#device = torch.device('cpu')
device = torch.device('cuda') # Uncomment this to run on GPU

In [75]:
x.device

device(type='cpu')

In [76]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
    x = x.to(device)                       # or just use strings ``.to("cuda")``
    z = x + y
    print(z,z.type())
    print(z.to("cpu", torch.double))       # ``.to`` can also change dtype together!

In [77]:
x = torch.randn(1)
x = x.to(device)

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [78]:
x.device

device(type='cpu')

In [80]:
# the following line is only useful if CUDA is available
x = x.data
print(x)
print(x.item())
print(x.cpu().numpy())

tensor([0.6497])
0.6497234106063843
[0.6497234]


# Simple interfaces to standard image data-bases

[Video timestamp](https://youtu.be/BmAS8IH7n3c?t=1354)

An example, the [CIFAR10](https://pytorch.org/docs/stable/torchvision/datasets.html#torchvision.datasets.CIFAR10) dataset.

In [81]:
import torchvision

data_dir = 'content/data'

cifar = torchvision.datasets.CIFAR10(data_dir, train = True, download = True)
cifar.data.shape

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting content/data\cifar-10-python.tar.gz to content/data


(50000, 32, 32, 3)

Documentation about the [`permute`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.permute) operation.

In [ ]:
x = torch.from_numpy(cifar.data).permute(0,3,1,2).float()
x = x / 255
print(x.type(), x.size(), x.min().item(), x.max().item())

Documentation about the [`narrow(input, dim, start, length)`](https://pytorch.org/docs/stable/torch.html#torch.narrow) operation.

In [ ]:
# Narrows to the first images, converts to float
x = torch.narrow(x, 0, 0, 48)

In [ ]:
x.shape

In [ ]:
# Showing images
def show(img):
    npimg = img.numpy()
    plt.figure(figsize=(20,10))
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')
    
show(torchvision.utils.make_grid(x, nrow = 12))

In [ ]:
# Kills the green and blue channels
x.narrow(1, 1, 2).fill_(0)
show(torchvision.utils.make_grid(x, nrow = 12))

# Autograd: automatic differentiation

[Video timestamp](https://youtu.be/Z6H3zakmn6E?t=40)

When executing tensor operations, PyTorch can automatically construct on-the-fly the graph of operations to compute the gradient of any quantity with respect to any tensor involved.

To be more concrete, we introduce the following example: we consider parameters $w\in \mathbb{R}$ and $b\in \mathbb{R}$ with the corresponding function:
\begin{eqnarray*}
\ell = \left(\exp(wx+b) - y^* \right)^2
\end{eqnarray*}

Our goal here, will be to compute the following partial derivatives:
\begin{eqnarray*}
\frac{\partial \ell}{\partial w}\mbox{ and, }\frac{\partial \ell}{\partial b}.
\end{eqnarray*}

The reason for doing this will be clear when you will solve the practicals for this lesson!

You can decompose this function as a composition of basic operations. This is call the forward pass on the graph of operations.
![backprop1](https://dataflowr.github.io/notebooks/Module2/img/backprop1.png)

Let say we start with our model in `numpy`:

In [82]:
w = np.array([0.5])
b = np.array([2])
xx = np.array([0.5])#np.arange(0,1.5,.5)

transform these into `tensor`:

In [83]:
xx_t = torch.from_numpy(xx)
w_t = torch.from_numpy(w)
b_t = torch.from_numpy(b)

[Video timestamp](https://youtu.be/Z6H3zakmn6E?t=224)

A `tensor` has a Boolean field `requires_grad`, set to `False` by default, which states if PyTorch should build the graph of operations so that gradients with respect to it can be computed.

In [88]:
w_t.requires_grad

True

We want to take derivative with respect to $w$ so we change this value:

In [89]:
w_t.requires_grad_(True)

tensor([0.5000], dtype=torch.float64, requires_grad=True)

We want to do the same thing for $b$ but the following line will produce an error!

In [95]:
b_t.requires_grad_(True)

tensor([2.], dtype=torch.float64, requires_grad=True)

Reading the error message should allow you to correct the mistake!

In [92]:
dtype = torch.float64

In [93]:
b_t = b_t.type(dtype)

In [94]:
b_t.requires_grad_(True)

tensor([2.], dtype=torch.float64, requires_grad=True)

[Video timestamp](https://youtu.be/Z6H3zakmn6E?t=404)

We now compute the function:

In [101]:
def fun(x,ystar):
    y = torch.exp(w_t*x+b_t)
    print(y)
    return torch.sum((y-ystar)**2)

ystar_t = torch.randn_like(xx_t)
l_t = fun(xx_t,ystar_t)

tensor([9.4877], dtype=torch.float64, grad_fn=<ExpBackward0>)


In [97]:
l_t

tensor(83.5964, dtype=torch.float64, grad_fn=<SumBackward0>)

In [98]:
l_t.requires_grad

True

After the computation is finished, i.e. *forward pass*, you can call ```.backward()``` and have all the gradients computed automatically.

In [102]:
print(w_t.grad)

None


In [105]:
l_t.backward()

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [104]:
print(w_t.grad)
print(b_t.grad)

tensor([79.4625], dtype=torch.float64)
tensor([158.9250], dtype=torch.float64)


[Video timestamp](https://youtu.be/Z6H3zakmn6E?t=545)

Let's try to understand these numbers...

![backprop2](https://dataflowr.github.io/notebooks/Module2/img/backprop2.png)

In [106]:
yy_t = torch.exp(w_t*xx_t+b_t)
print(torch.sum(2*(yy_t-ystar_t)*yy_t*xx_t))
print(torch.sum(2*(yy_t-ystar_t)*yy_t))

tensor(79.4625, dtype=torch.float64, grad_fn=<SumBackward0>)
tensor(158.9250, dtype=torch.float64, grad_fn=<SumBackward0>)


`tensor.backward()` accumulates the gradients in  the `grad` fields  of tensors.

In [107]:
l_t = fun(xx_t,ystar_t)
l_t.backward()

tensor([9.4877], dtype=torch.float64, grad_fn=<ExpBackward0>)


In [108]:
print(w_t.grad)
print(b_t.grad)

tensor([158.9250], dtype=torch.float64)
tensor([317.8500], dtype=torch.float64)


By default, `backward` deletes the computational graph when it is used so that you will get an error below:

In [109]:
l_t.backward()

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [110]:
# Manually zero the gradients
w_t.grad.data.zero_()
b_t.grad.data.zero_()
l_t = fun(xx_t,ystar_t)
l_t.backward(retain_graph=True)
l_t.backward()
print(w_t.grad)
print(b_t.grad)

tensor([9.4877], dtype=torch.float64, grad_fn=<ExpBackward0>)
tensor([158.9250], dtype=torch.float64)
tensor([317.8500], dtype=torch.float64)


The gradients must be set to zero manually. Otherwise they will cumulate across several _.backward()_ calls. 
This accumulating behavior is desirable in particular to compute the gradient of a loss summed over several “mini-batches,” or the gradient of a sum of losses.

[![Dataflowr](https://raw.githubusercontent.com/dataflowr/website/master/_assets/dataflowr_logo.png)](https://dataflowr.github.io/website/)